### 🖼️ Función para cargar imagen en RGB y escala de grises

Esta función `cargar_imagen` recibe la ruta de una imagen como entrada y devuelve dos versiones de la imagen:

1. `img_rgb`: Imagen convertida al espacio de color RGB, ideal para mostrarla con `matplotlib`, ya que OpenCV carga las imágenes en formato BGR por defecto.
2. `img_gray`: Imagen convertida a escala de grises, que es útil para aplicar filtros como el gaussiano (más eficiente en una sola capa de intensidad que en las 3 del color).

También incluye una verificación para asegurarse de que la imagen fue cargada correctamente.

In [ ]:
import cv2
import numpy as np
import os
from utils import *
import matplotlib.pyplot as plt

### 🧮 Filtro de paso alto utilizando suavizado con la Media

La función `filtro_paso_alto_media` aplica un filtro de paso alto a una imagen en escala de grises utilizando un suavizado mediante la media (también conocido como filtro de caja o **box blur**).

**¿Cómo funciona?**
- Se suaviza la imagen aplicando un filtro de la media con un kernel cuadrado de tamaño `ksize`.
- Luego, se resta esta imagen suavizada de la original. Esta operación elimina las zonas de baja frecuencia (regiones uniformes) y destaca las altas frecuencias (como bordes y cambios bruscos de intensidad).

Este método es una alternativa simple al filtro Gaussiano para destacar detalles en imágenes, y suele ser computacionalmente más ligero, aunque menos preciso en la preservación de bordes suaves.


In [ ]:
def filtro_paso_alto_media(img_gray, ksize=5):
    suavizada = cv2.blur(img_gray, (ksize, ksize))
    return cv2.subtract(img_gray, suavizada)

### 🗂️ Procesamiento masivo de imágenes con filtro de la Media

Esta función `procesar_imagenes_carpeta_media` aplica el filtro de paso alto basado en la media (box blur) a todas las imágenes `.png` dentro de una estructura de carpetas.

**Qué hace:**
- Recorre subcarpetas dentro de una carpeta principal.
- Aplica el filtro de paso alto utilizando el suavizado por media a cada imagen.
- Guarda las imágenes resultantes respetando la misma estructura de carpetas.
- Calcula la varianza de cada imagen procesada y la guarda en un archivo `.txt`.

**Utilidad:**
Automatiza el análisis de contenido de alta frecuencia usando un filtro más simple y rápido que el Gaussiano, lo cual es útil para tareas preliminares de detección de bordes, análisis de textura o comparación entre filtros.


In [ ]:
def procesar_imagenes_carpeta_media(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
    ksize=5
):
    os.makedirs(carpeta_salida, exist_ok=True)

    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)

        if not os.path.isdir(ruta_subcarpeta):
            continue

        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta, "media")
        os.makedirs(carpeta_salida_sub, exist_ok=True)

        ruta_txt = os.path.join(carpeta_salida_sub, "info_varianzas_media.txt")
        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Varianzas de alta frecuencia (filtro de la media)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("La varianza indica la dispersión de los valores de alta frecuencia.\n")
            archivo_txt.write("Valores mayores suelen corresponder a más bordes y mayor nitidez.\n\n")

            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if not filename.lower().endswith(".png"):
                    continue

                ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                img_bgr = cv2.imread(ruta_imagen_entrada)
                if img_bgr is None:
                    continue

                img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

                # Aplicar filtro y calcular varianza en una sola vez
                altas_frec_img = filtro_paso_alto_media(img_gray, ksize=ksize)
                varianza = np.var(altas_frec_img)

                # Normalizar para guardar con buen contraste
                hf_min, hf_max = altas_frec_img.min(), altas_frec_img.max()
                if hf_max > hf_min:
                    altas_frec_escalada = ((altas_frec_img - hf_min) / (hf_max - hf_min) * 255).astype(np.uint8)
                else:
                    altas_frec_escalada = altas_frec_img.astype(np.uint8)

                nombre, _ = os.path.splitext(filename)
                nombre_salida = f"{nombre}_alta_frecuencia_media.png"
                ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)

                cv2.imwrite(ruta_imagen_salida, altas_frec_escalada)
                archivo_txt.write(f"{filename} -> varianza: {varianza:.3f}\n")

    print("Procesamiento completado con filtro de la Media.")


In [ ]:
# Ejecuta el procesamiento
procesar_imagenes_carpeta_media(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
    ksize=5
)